In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['Qtyeach'] = input_['Qtyeach'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：10


['CCR-3',
 'CCR-6',
 'CCR-7',
 'DS-2282',
 'LS-312',
 'NS-35',
 'NS-56',
 'NS-63',
 'NS-68',
 'NS-69']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?partNum={input_.loc[a, 'Part Number']}&brand={input_.loc[a, 'Brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960'''
        check_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
dict_src, list_src = {}, resp.json()
for i, src in enumerate(list_src):
    dict_src[str(i)] = src['image_BG_Url'].strip()

dict_src

{'0': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=lyZLKcNT6hOguYlO0X7TiSYF4nkLByFFki+PA/U85obZF1f8c92iBGL32PrFFzjl5VIjH+toj7m4MFyxxPbcYCb9GMryv61nnF+g5s4yuSwDxoJ3wM6CXnojbpNR9ZyLIIhFT3wvlhQYiGmyzTdB6KnruYWZQkXzOw3ACMBE4NOl7YolOiMBrRlx4ftRt81KnB2R+IpAIPQ=',
 '1': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=lyZLKcNT6hOguYlO0X7TiSYF4nkLByFFki+PA/U85obZF1f8c92iBI7EBIbNw+Iv5VIjH+toj7m4MFyxxPbcYCb9GMryv61nnF+g5s4yuSwDxoJ3wM6CXnojbpNR9ZyLIIhFT3wvlhQYiGmyzTdB6KnruYWZQkXzOw3ACMBE4NOl7YolOiMBrRlx4ftRt81KnB2R+IpAIPQ=',
 '2': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=lyZLKcNT6hOguYlO0X7TiSYF4nkLByFFki+PA/U85obZF1f8c92iBH5GBWYcbQEZ5VIjH+toj7m4MFyxxPbcYCb9GMryv61nnF+g5s4yuSwDxoJ3wM6CXnojbpNR9ZyLIIhFT3wvlhQYiGmyzTdB6KnruYWZQkXzOw3ACMBE4NOl7YolOiMBrRlx4ftRt81KnB2R+IpAIPQ='}

In [9]:
import json

df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
df_temp.loc[0, 'Picture'] = ''
df_temp.loc[0, 'Json_Src'] = json.dumps(dict_src)

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec,Picture,Json_Src
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...,Kia Sephia (01-94)\nKia Spectra (04-00)\nKia S...,1. Fuel Filter Flush (Required)\n2. Oil Change...,STD,CCR3,STI,Clutch Starter Safety / Cruise Release Switch,Clutch Starter Safety / Cruise Release Switch,F,1,"{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""...",,"{""0"": ""https://ecatalog.smpcorp.com/ihgs/image..."


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_src.xlsx', index=False)

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec,Picture,Json_Src
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...,Kia Sephia (01-94)\nKia Spectra (04-00)\nKia S...,1. Fuel Filter Flush (Required)\n2. Oil Change...,STD,CCR3,STI,Clutch Starter Safety / Cruise Release Switch,Clutch Starter Safety / Cruise Release Switch,F,1,"{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""...",,"{""0"": ""https://ecatalog.smpcorp.com/ihgs/image..."


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_src_error.xlsx', index=False)

output_error

""
